# Assignment - MMB
*Alexander Laloi Dybdahl, Valentin Vuillon, Alexia Stéphanie Liviana Paratte*

In [1]:
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models, tools
from biogeme.expressions import Beta, DefineVariable, Variable, bioDraws, MonteCarlo, log, Power, exp, Derive
import scipy.stats as st


### Loading data

In [2]:
df = pd.read_csv("lpmc07.dat", delimiter='\t')

In [18]:
df.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent', 'dur_pt_total',
       'cost_driving_total', 'car_available'],
      dtype='object')

## Tasks

### Model 0

Model 0 includes a general cost parameter and alternative-specific time parameters for each mode of transportation. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC\_WALK} + \beta_{\text{TIME}} \cdot \text{dur\_walking} + \epsilon_{\text{walk}} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC\_BIKE} + \beta_{\text{TIME}} \cdot \text{dur\_cycling} + \epsilon_{\text{cycle}} $$

- **Public Transport**:  
  $$ U_{\text{pt}} = \text{ASC\_PT} + \beta_{\text{COST}} \cdot \text{cost\_transit} + \beta_{\text{TIME}} \cdot \text{dur\_pt\_total} + \epsilon_{\text{pt}} $$

- **Driving**:  
  $$ U_{\text{drive}} = \text{ASC\_DRIVE} + \beta_{\text{COST}} \cdot \text{cost\_driving\_total} + \beta_{\text{TIME}} \cdot \text{dur\_driving} + \epsilon_{\text{drive}} $$

where:
- $ \beta_{\text{COST}} $ is the coefficient for travel cost.
- $ \beta_{\text{TIME}} $ is the coefficient for travel time.
- $ \text{cost}_j $ is the travel cost for mode $ j $.
- $ \text{dur}_j $ is the travel time for mode $ j $.
- $ \epsilon_j $ is the error term, representing unobserved factors affecting the utility of mode $ j $.

The probability $ P_j $ of choosing mode $ j $ is given by the softmax function:

$$ P_j = \frac{\exp(U_j)}{\sum_{k=1}^{J} \exp(U_k)} $$


In [3]:
# Calculate the total public transport duration and total driving cost
df['dur_pt_total'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_bus'] + df['dur_pt_int']
df['cost_driving_total'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']
df['car_available'] = (df['car_ownership'] > 0).astype(int)

# Create a Biogeme database
database = db.Database('LPMC', df)
globals().update(database.variables)

# Define parameters for the utility functions
ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)

BETA_COST = Beta('BETA_COST', 0, None, None, 0)
BETA_TIME = Beta('BETA_TIME', 0, None, None, 0)

# Define utility functions using Biogeme expressions
V1 = ASC_WALK + BETA_TIME * dur_walking
V2 = ASC_BIKE + BETA_TIME * dur_cycling
V3 = ASC_PT + BETA_COST * cost_transit + BETA_TIME * dur_pt_total
V4 = BETA_COST * cost_driving_total + BETA_TIME * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_0 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_0 = bio.BIOGEME(database, logprob_0)
biogeme_0.modelName = 'Model_0'
biogeme_0.generateHtml = False  # Disable HTML file generation
biogeme_0.generatePickle = False  # Disable PICKLE file generation
biogeme_0.save_iterations = False  # Disable ITER file generation
results_model_0 = biogeme_0.estimate()

# Output
print(results_model_0.getEstimatedParameters())

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


              Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE  -2.569395      0.090262   -28.466003           0.0
ASC_PT     0.766417      0.047360    16.182870           0.0
ASC_WALK   1.256090      0.076712    16.374060           0.0
BETA_COST -0.173019      0.014562   -11.881537           0.0
BETA_TIME -5.326766      0.189549   -28.102283           0.0


In [4]:
# Retrieve the general statistics from the results
general_stats_model_0 = results_model_0.getGeneralStatistics()
print(results_model_0.printGeneralStatistics())


Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4642.324
Likelihood ratio test for the init. model:	4578.295
Rho-square for the init. model:	0.33
Rho-square-bar for the init. model:	0.33
Akaike Information Criterion:	9294.648
Bayesian Information Criterion:	9327.234
Final gradient norm:	2.0414E-04
Nbr of threads:	8



### Model 1

Model 1 includes alternative-specific cost parameters for each mode of transportation. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC\_WALK} + \beta_{\text{TIME\_WALK}} \cdot \text{dur\_walking} + \epsilon_{\text{walk}} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC\_BIKE} + \beta_{\text{TIME\_BIKE}} \cdot \text{dur\_cycling} + \epsilon_{\text{cycle}} $$

- **Public Transport**:  
  $$ U_{\text{pt}} = \text{ASC\_PT} + \beta_{\text{COST}} \cdot \text{cost\_transit} + \beta_{\text{TIME\_PT}} \cdot \text{dur\_pt\_total} + \epsilon_{\text{pt}} $$

- **Driving**:  
  $$ U_{\text{drive}} = \beta_{\text{COST}} \cdot \text{cost\_driving\_total} + \beta_{\text{TIME\_DRIVE}} \cdot \text{dur\_driving} + \epsilon_{\text{drive}} $$

Where:
- $ \text{ASC\_WALK}, \text{ASC\_BIKE}, \text{ASC\_PT} $ are the alternative specific constants for walking, cycling, and public transport, respectively.
- $ \beta_{\text{COST}} $ is the common cost coefficient for all transportation modes.
- $ \beta_{\text{TIME\_WALK}}, \beta_{\text{TIME\_BIKE}}, \beta_{\text{TIME\_PT}}, \beta_{\text{TIME\_DRIVE}} $ are the time coefficients for walking, cycling, public transport, and driving, respectively.
- $ \text{cost\_transit}, \text{cost\_driving\_total} $ are the costs associated with public transport and driving.
- $ \text{dur\_walking}, \text{dur\_cycling}, \text{dur\_pt\_total}, \text{dur\_driving} $ are the travel durations for each mode.


In [5]:

# Define additional parameters for the cost for each mode
BETA_TIME_WALK = Beta('BETA_TIME_WALK', 0, None, None, 0)
BETA_TIME_BIKE = Beta('BETA_TIME_BIKE', 0, None, None, 0)
BETA_TIME_PT = Beta('BETA_TIME_PT', 0, None, None, 0)
BETA_TIME_DRIVE = Beta('BETA_TIME_DRIVE', 0, None, None, 0)

# Define utility functions using Biogeme expressions with alternative-specific cost coefficients
V1 = ASC_WALK + BETA_TIME_WALK * dur_walking
V2 = ASC_BIKE + BETA_TIME_BIKE * dur_cycling
V3 = ASC_PT + BETA_COST * cost_transit + BETA_TIME_PT * dur_pt_total
V4 = BETA_COST * cost_driving_total + BETA_TIME_DRIVE * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_1 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_1 = bio.BIOGEME(database, logprob_1)
biogeme_1.modelName = 'Model_1'
biogeme_1.generateHtml = False  # Disable HTML file generation
biogeme_1.generatePickle = False  # Disable PICKLE file generation
biogeme_1.save_iterations = False  # Disable ITER file generation
results_model_1 = biogeme_1.estimate()

# Output
print(results_model_1.getEstimatedParameters())

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


                    Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE        -2.330094      0.157067   -14.835060      0.000000
ASC_PT          -0.321680      0.068372    -4.704883      0.000003
ASC_WALK         2.078816      0.134682    15.434943      0.000000
BETA_COST       -0.163540      0.016154   -10.123734      0.000000
BETA_TIME_BIKE  -6.714893      0.589110   -11.398368      0.000000
BETA_TIME_DRIVE -5.841121      0.364543   -16.023120      0.000000
BETA_TIME_PT    -3.291291      0.235273   -13.989234      0.000000
BETA_TIME_WALK  -8.487446      0.409809   -20.710732      0.000000


**Alternative Specific Constants (ASCs):**

- $ \text{ASC}_{\text{bike}}, \text{ASC}_{\text{pt}}, \text{and} \text{ASC}_{\text{walk}} $ are statistically significant with near-zero p-values. The constants indicate a baseline aversion to cycling ($ \text{ASC}_{\text{bike}} < 0 $) and a preference for walking, driving, and public transport ($ \text{ASC}_{\text{walk}}, \text{ASC}_{\text{drive}}, \text{ASC}_{\text{pt}} > 0 $).

**Alternative-Specific Cost Coefficients:**

- $ \beta_{\text{cost\_bike}} $ and $ \beta_{\text{cost\_walk}} $ are zero, indicating no significant impact of costs on biking and walking utility.
- $ \beta_{\text{cost\_drive}} $ is negative and significant, suggesting higher driving costs reduce its utility.
- $ \beta_{\text{cost\_pt}} $ is positive and significant, an unexpected result implying higher public transport costs might correlate with higher utility, potentially reflecting unmodeled factors like income or service quality.

**Time Coefficient ($ \beta_{\text{time}} $):**

- Remains negative and significant, indicating longer travel times decrease the utility of a mode.


### Comparing $\text{Model 1}$ and Model 0

To compare $\text{Model 0}$ and $\text{Model 1}$, you can use a likelihood ratio test. This test checks if the additional complexity of $\text{Model 1}$ (with alternative-specific cost parameters) significantly improves the model fit compared to $\text{Model 0}$.

- **Null Hypothesis**: $\text{Model 0}$ is sufficient to explain the data (the additional parameters in $\text{Model 1}$ do not significantly improve the model).

- **Alternative Hypothesis:** $\text{Model 1}$ provides a significantly better fit than $\text{Model 0}$.

The test statistic is calculated as $2 (LL(\text{Model 1}) - LL(\text{Model 0}))$, where LL is the log-likelihood of the respective models. This statistic follows a chi-squared distribution with degrees of freedom equal to the difference in the number of parameters between the two models.

Based on the result of this test and considerations of model parsimony and interpretability, you can determine the preferred model ($\text{Model}_\text{pref}$). Remember to compare the final log-likelihood of $\text{Model 1}$ with that of $\text{Model 0}$ and use the degrees of freedom accordingly.

In [6]:
LR_test = 2 * (results_model_1.data.logLike - results_model_0.data.logLike)
print("Log likelihood ratio:", LR_test)
p_val = st.chi2.sf(LR_test, 4)
print("p value:", p_val)
x_qhi = st.chi2.ppf(0.05, 4)
print("Critical value:", x_qhi)

# Get general statistics for Model 1
general_stats_model_1 = results_model_1.getGeneralStatistics()
print(results_model_1.printGeneralStatistics())

Log likelihood ratio: 626.285269855598
p value: 3.1696293841371207e-134
Critical value: 0.7107230213973239
Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4329.182
Likelihood ratio test for the init. model:	5204.58
Rho-square for the init. model:	0.375
Rho-square-bar for the init. model:	0.374
Akaike Information Criterion:	8674.363
Bayesian Information Criterion:	8726.501
Final gradient norm:	2.5479E-03
Nbr of threads:	8



#### Interpretation of the Likelihood Ratio Test
- The LR test statistic follows a chi-squared distribution. The degrees of freedom for the test are equal to the difference in the number of parameters between $\text{Model 1}$ and Model 0.

- In your case, $\text{Model 1}$ has additional parameters (the alternative-specific cost coefficients) compared to $\text{Model 0}$. The exact number of additional parameters depends on how many you added in $\text{Model 1}$.

#### Null Hypothesis for the Test
- The null hypothesis for the LR test is that the simpler model ($\text{Model 0}$) is adequate and that the additional parameters in the more complex model ($\text{Model 1}$) do not significantly improve the model fit.

#### Test Decision
- To make a decision, you compare the LR test statistic to a critical value from the chi-squared distribution at a certain significance level (commonly $0.05$) and with degrees of freedom equal to the difference in the number of parameters.
- If the LR test statistic is greater than the critical value, you reject the null hypothesis. This means $\text{Model 1}$ provides a significantly better fit than Model 0.

#### Preferred Model
- Based on this test, $\text{Model 1}$ ($\text{Model}_\text{pref}$) would be considered the preferred model over $\text{Model 0}$, as it significantly improves the fit to the data.
- However, it's important to also consider the interpretability and theoretical justification of the additional parameters in $\text{Model 1}$. Sometimes a more complex model is not preferable if it does not add meaningful explanatory power or if it makes the model less interpretable.

### Model 2

Model 2 introduces interactions with the socio-economic characteristic variable $\text{car\_available}$, building upon the specifications from $\text{Model\_pref}$. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC\_WALK} \cdot \text{car\_available} + \text{ASC\_WALK\_NOCAR} \cdot (1 - \text{car\_available}) + \beta_{\text{TIME\_WALK}} \cdot \text{dur\_walking} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC\_BIKE} \cdot \text{car\_available} + \text{ASC\_BIKE\_NOCAR} \cdot (1 - \text{car\_available}) + \beta_{\text{TIME\_BIKE}} \cdot \text{dur\_cycling} $$

- **Public Transport**: 
  $$ U_{\text{pt}} = \text{ASC\_PT} \cdot \text{car\_available} + \text{ASC\_PT\_NOCAR} \cdot (1 - \text{car\_available}) + \beta_{\text{COST}} \cdot \text{cost\_transit} + \beta_{\text{TIME\_PT}} \cdot \text{dur\_pt\_total} $$

- **Driving**:  
  $$ U_{\text{drive}} = \beta_{\text{COST}} \cdot \text{cost\_driving\_total} + \beta_{\text{TIME\_DRIVE}} \cdot \text{dur\_driving} $$

Where:
- $\text{ASC\_WALK}, \text{ASC\_BIKE}, \text{ASC\_PT}$ are the alternative specific constants.
- $\text{ASC\_WALK\_NOCAR}, \text{ASC\_BIKE\_NOCAR}, \text{ASC\_PT\_NOCAR}$ are the constants for when $\text{car\_available}$ is 0.
- $\beta_{\text{COST}}$ is the cost coefficient applicable to public transport and driving.
- $\beta_{\text{TIME\_WALK}}, \beta_{\text{TIME\_BIKE}}, \beta_{\text{TIME\_PT}}, \beta_{\text{TIME\_DRIVE}}$ are the time coefficients for each mode.
- $\text{car\_available}$ is a binary variable indicating car availability.


In [7]:

# Define additional parameters for the utility functions
ASC_WALK_NOCAR = Beta('ASC_WALK_NOCAR', 0, None, None, 0)
ASC_BIKE_NOCAR = Beta('ASC_BIKE_NOCAR', 0, None, None, 0)
ASC_PT_NOCAR = Beta('ASC_PT_NOCAR', 0, None, None, 0)

# Utility functions with interactions
V1 = ASC_WALK * car_available + ASC_WALK_NOCAR * (1 - car_available) + BETA_TIME_WALK * dur_walking
V2 = ASC_BIKE * car_available + ASC_BIKE_NOCAR * (1 - car_available) + BETA_TIME_BIKE * dur_cycling
V3 = ASC_PT * car_available + ASC_PT_NOCAR * (1 - car_available) + BETA_COST * cost_transit + BETA_TIME_PT * dur_pt_total
V4 = BETA_COST * cost_driving_total + BETA_TIME_DRIVE * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_2 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_2 = bio.BIOGEME(database, logprob_2)
biogeme_2.modelName = 'Model_2'
biogeme_2.generateHtml = False  # Disable HTML file generation
biogeme_2.generatePickle = False  # Disable PICKLE file generation
biogeme_2.save_iterations = False  # Disable ITER file generation
results_model_2 = biogeme_2.estimate()

# Output
print(results_model_2.getEstimatedParameters())


Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


                    Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE        -2.756434      0.172031   -16.022902      0.000000
ASC_BIKE_NOCAR  -0.212168      0.212117    -1.000239      0.317195
ASC_PT          -1.115912      0.083781   -13.319371      0.000000
ASC_PT_NOCAR     2.108649      0.129278    16.310932      0.000000
ASC_WALK         1.531068      0.138406    11.062132      0.000000
ASC_WALK_NOCAR   4.270330      0.184426    23.154767      0.000000
BETA_COST       -0.172165      0.018578    -9.267241      0.000000
BETA_TIME_BIKE  -6.823722      0.596936   -11.431239      0.000000
BETA_TIME_DRIVE -5.912840      0.375662   -15.739779      0.000000
BETA_TIME_PT    -3.215695      0.259893   -12.373141      0.000000
BETA_TIME_WALK  -8.443463      0.414100   -20.389902      0.000000


**Alternative Specific Constants (ASCs):**

- $ \text{ASC}_{\text{bike}}, \text{ASC}_{\text{drive}}, \text{ASC}_{\text{pt}}, \text{and } \text{ASC}_{\text{walk}} $ are all statistically significant with p-values close to zero.

**Cost Coefficients:**

- $ \beta_{\text{cost\_bike}} $ and $ \beta_{\text{cost\_walk}} $ are zero, implying that costs do not significantly influence the utility of biking and walking.
- $ \beta_{\text{cost\_drive}} $ is negative and significant, meaning higher driving costs reduce its utility.
- $ \beta_{\text{cost\_pt}} $ is positive but not statistically significant, indicating that cost variations in public transport do not substantially affect its utility.

**Interaction Terms:**

- $ \beta_{\text{drive\_carown}} $ is positive and significant, highlighting that car ownership notably increases the utility of driving.
- $ \beta_{\text{cost\_drive\_carown}} $, while positive, is not statistically significant, suggesting that the interaction effect of driving costs and car ownership on driving utility is indeterminate in this model.

**Time Coefficient:**

- $ \beta_{\text{time}} $ remains negative and significant, reinforcing that longer travel times reduce the utility of all modes.

**Interpretation and Implications:**

- The negative $ \text{ASC}_{\text{drive}} $ reflects a shift in baseline driving preference when considering car ownership, underscored by the significant positive interaction with car ownership.
- The positive and significant $ \beta_{\text{drive\_carown}} $ aligns with the intuitive expectation that car ownership increases the utility of driving.
- The indistinct impact of $ \beta_{\text{cost\_drive\_carown}} $ suggests that car owners' sensitivity to driving costs may not differ notably from non-owners in this dataset.
- The zero coefficients for $ \beta_{\text{cost\_bike}} $ and $ \beta_{\text{cost\_walk}} $ continue to indicate that cost is not a pivotal factor in the choice to walk or cycle.


In [19]:
Model 2 introduces interactions with the socio-economic characteristic variable $\text{car\_available}$, building upon the specifications from $\text{Model\_pref}$. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC\_WALK} \cdot \text{car\_available} + \text{ASC\_WALK\_NOCAR} \cdot (1 - \text{car\_available}) + \beta_{\text{TIME\_WALK}} \cdot \text{dur\_walking} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC\_BIKE} \cdot \text{car\_available} + \text{ASC\_BIKE\_NOCAR} \cdot (1 - \text{car\_available}) + \beta_{\text{TIME\_BIKE}} \cdot \text{dur\_cycling} $$

- **Public Transport**: 
  $$ U_{\text{pt}} = \text{ASC\_PT} \cdot \text{car\_available} + \text{ASC\_PT\_NOCAR} \cdot (1 - \text{car\_available}) + \beta_{\text{COST}} \cdot \text{cost\_transit} + \beta_{\text{TIME\_PT}} \cdot \text{dur\_pt\_total} $$

- **Driving**:  
  $$ U_{\text{drive}} = \beta_{\text{COST}} \cdot \text{cost\_driving\_total} + \beta_{\text{TIME\_DRIVE}} \cdot \text{dur\_driving} $$

Where:
- $\text{ASC\_WALK}, \text{ASC\_BIKE}, \text{ASC\_PT}$ are the alternative specific constants.
- $\text{ASC\_WALK\_NOCAR}, \text{ASC\_BIKE\_NOCAR}, \text{ASC\_PT\_NOCAR}$ are the constants for when $\text{car\_available}$ is 0.
- $\beta_{\text{COST}}$ is the cost coefficient applicable to public transport and driving.
- $\beta_{\text{TIME\_WALK}}, \beta_{\text{TIME\_BIKE}}, \beta_{\text{TIME\_PT}}, \beta_{\text{TIME\_DRIVE}}$ are the time coefficients for each mode.
- $\text{car\_available}$ is a binary variable indicating car availability.

SyntaxError: invalid syntax (2749452522.py, line 1)

### Comparing Model 2 and $\text{Model 1}$

**Model Comparison ($\text{Model}_\text{pref}$ vs. $\text{Model 2}$):**
To compare $\text{Model 2}$ with $\text{Model}_\text{pref}$, you can use a likelihood ratio test:

- **Null Hypothesis:** $\text{Model}_\text{pref}$ is sufficient, and the additional interaction terms in $\text{Model 2}$ do not significantly improve the model.
- **Alternative Hypothesis:** $\text{Model 2}$ provides a significantly better fit than $\text{Model}_\text{pref}$.

Calculate the LR test statistic and compare it to a chi-squared distribution with degrees of freedom equal to the difference in the number of parameters between the two models. The decision on the preferred model should consider both statistical significance and the interpretability of the model.

In [8]:
LR_test = 2 * (results_model_2.data.logLike - results_model_1.data.logLike)
print("Log likelihood ratio test:", LR_test)
x_qhi = st.chi2.sf(LR_test, 2)
print("Critical value:", x_qhi)

# Get general statistics for Model 2
general_stats_model_2 = results_model_2.getGeneralStatistics()
print(results_model_2.printGeneralStatistics())

Log likelihood ratio test: 1230.4890522020705
Critical value: 6.348882552082338e-268
Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-3713.937
Likelihood ratio test for the init. model:	6435.07
Rho-square for the init. model:	0.464
Rho-square-bar for the init. model:	0.463
Akaike Information Criterion:	7449.874
Bayesian Information Criterion:	7521.563
Final gradient norm:	7.1265E-04
Nbr of threads:	8




- Calculated LR test statistic:
- **Interpretation**:
  - The high value of the LR test statistic suggests that $\text{Model 2}$ provides a significantly better fit to the data compared to $\text{Model 1}$.
- **Test Decision**:
  - With a large LR statistic, the null hypothesis (that $\text{Model 1}$ is sufficient) is likely rejected, indicating a preference for $\text{Model 2}$.
- **Conclusion**:
  - $\text{Model 2}$, with its additional parameters and interactions, is the preferred model over $\text{Model 1}$, given its significantly better fit to the data.


### Model 3

Model 3 integrates a non-linear transformation (specifically, a boxcox transformation) for travel durations in its utility functions. The utility functions are now defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC\_WALK} \cdot \text{car\_available} + \text{ASC\_WALK\_NOCAR} \cdot (1 - \text{car\_available}) + \beta_{\text{TIME\_WALK}} \cdot \text{dur\_walking}(\lambda) + \epsilon_{walk} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC\_BIKE} \cdot \text{car\_available} + \text{ASC\_BIKE\_NOCAR} \cdot (1 - \text{car\_available}) + \beta_{\text{TIME\_BIKE}} \cdot \text{dur\_cycling}(\lambda) + \epsilon_{cycle} $$

- **Public Transport**: 
  $$ U_{\text{pt}} = \text{ASC\_PT} \cdot \text{car\_available} + \text{ASC\_PT\_NOCAR} \cdot (1 - \text{car\_available}) + \beta_{\text{COST}} \cdot \text{cost\_transit} + \beta_{\text{TIME\_PT}} \cdot \text{dur\_pt\_total}(\lambda) + \epsilon_{pt} $$

- **Driving**:  
  $$ U_{\text{drive}} = \beta_{\text{COST}} \cdot \text{cost\_driving\_total} + \beta_{\text{TIME\_DRIVE}} \cdot \text{dur\_driving}(\lambda) + \epsilon_{driving} $$

Where:
- $dur_i(\lambda)= 
    \begin{cases}
        \frac{dur_i^{\lambda}-1}{\lambda},& \text{if } \lambda \neq 0\\
        \ln dur_i,              & \lambda = 0
    \end{cases}$ 
    with $i \in \{$walk, cycle, pt\_total, drive$\}$
- $\text{ASC\_WALK}, \text{ASC\_BIKE}, \text{ASC\_PT}$ are the alternative specific constants.
- $\text{ASC\_WALK\_NOCAR}, \text{ASC\_BIKE\_NOCAR}, \text{ASC\_PT\_NOCAR}$ are constants for when $\text{car\_available}$ is 0.
- $\beta_{\text{COST}}$ is the cost coefficient applicable to public transport and driving.
- $\beta_{\text{TIME\_WALK}}, \beta_{\text{TIME\_BIKE}}, \beta_{\text{TIME\_PT}}, \beta_{\text{TIME\_DRIVE}}$ are the coefficients for the logarithmic transformation of the travel durations for each mode.
- $\text{car\_available}$ is a binary variable indicating car availability.


In [9]:

# Define boxcox parameters
lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0)
boxcox_time_1 = models.boxcox(dur_walking, lambda_boxcox)
boxcox_time_2 = models.boxcox(dur_cycling, lambda_boxcox)
boxcox_time_3 = models.boxcox(dur_pt_total, lambda_boxcox)
boxcox_time_4 = models.boxcox(dur_driving, lambda_boxcox)

# Define utility function
V1_boxcox = ASC_WALK * car_available + ASC_WALK_NOCAR * (1 - car_available) + BETA_TIME_WALK * boxcox_time_1
V2_boxcox = ASC_BIKE * car_available + ASC_BIKE_NOCAR * (1 - car_available) + BETA_TIME_BIKE * boxcox_time_2
V3_boxcox = ASC_PT * car_available + ASC_PT_NOCAR * (1 - car_available) + BETA_COST * cost_transit + BETA_TIME_PT * boxcox_time_3
V4_boxcox = BETA_COST * cost_driving_total + BETA_TIME_DRIVE * boxcox_time_4

V_boxcox = {1: V1_boxcox, 2: V2_boxcox, 3: V3_boxcox, 4: V4_boxcox}

# Define the model
logprob_3 = models.loglogit(V_boxcox, None, travel_mode)

# Estimate the model
biogeme_3 = bio.BIOGEME(database, logprob_3)
biogeme_3.modelName = 'Model_3'
biogeme_3.generateHtml = False  # Disable HTML file generation
biogeme_3.generatePickle = False  # Disable PICKLE file generation
biogeme_3.save_iterations = False  # Disable ITER file generation
results_model_3 = biogeme_3.estimate()

# Output
print(results_model_3.getEstimatedParameters())



It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


                    Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE        -3.554901      0.267075   -13.310471  0.000000e+00
ASC_BIKE_NOCAR  -1.043633      0.308036    -3.388025  7.039794e-04
ASC_PT           0.960025      0.138845     6.914347  4.700240e-12
ASC_PT_NOCAR     4.149450      0.181947    22.805847  0.000000e+00
ASC_WALK        -0.789252      0.243585    -3.240147  1.194680e-03
ASC_WALK_NOCAR   1.851213      0.263292     7.031017  2.050360e-12
BETA_COST       -0.165091      0.018022    -9.160703  0.000000e+00
BETA_TIME_BIKE  -3.907777      0.360839   -10.829709  0.000000e+00
BETA_TIME_DRIVE -3.380950      0.303389   -11.143960  0.000000e+00
BETA_TIME_PT    -2.516708      0.205354   -12.255444  0.000000e+00
BETA_TIME_WALK  -5.690655      0.304760   -18.672587  0.000000e+00
lambda_boxcox    0.366260      0.056974     6.428515  1.288565e-10


In [10]:
# Get general statistics for Model 3
general_stats_model_3 = results_model_3.printGeneralStatistics()
print(general_stats_model_3)

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-3661.787
Likelihood ratio test for the init. model:	6539.369
Rho-square for the init. model:	0.472
Rho-square-bar for the init. model:	0.47
Akaike Information Criterion:	7347.575
Bayesian Information Criterion:	7425.781
Final gradient norm:	4.7343E-03
Nbr of threads:	8



### Model 4

#### Nesting

In [11]:

# Define nest coefficients
MOTOR = Beta('MOTOR', 1, 1, None, 0)  # Nest parameter for motorized transport
PRIVATIZED = Beta('PRIVATIZED', 1, 1, None, 0)  # Nest parameter for non-motorized transport

# Define nests
nest_motorized = MOTOR, {1: 0,
                         2: 0,
                         3: 1,
                         4: 1}
nest_privatized = PRIVATIZED, {1: 1,
                               2: 1,
                               3: 0,
                               4: 0}

# Combine nests into a list
nests = nest_motorized, nest_privatized

# Define the cross-nested logit model
nested_logit = models.logcnl(V, None, nests, travel_mode)

# Estimate the model
biogeme_4_nest = bio.BIOGEME(database, nested_logit)
biogeme_4_nest.modelName = 'Model_4_crossnest'
biogeme_4_nest.generateHtml = False  # Disable HTML file generation
biogeme_4_nest.generatePickle = False  # Disable PICKLE file generation
biogeme_4_nest.save_iterations = False  # Disable ITER file generation
results_model_4 = biogeme_4_nest.estimate()

# Print the estimation results
print(results_model_4.getEstimatedParameters())

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


                    Value  Active bound  Rob. Std err  Rob. t-test  \
ASC_BIKE        -2.756359           0.0      1.337969    -2.060106   
ASC_BIKE_NOCAR  -0.212104           0.0      0.919235    -0.230739   
ASC_PT          -1.115927           0.0      0.539788    -2.067344   
ASC_PT_NOCAR     2.108622           0.0      1.058003     1.993021   
ASC_WALK         1.531152           0.0      0.384378     3.983451   
ASC_WALK_NOCAR   4.270413           0.0      1.237604     3.450547   
BETA_COST       -0.172168           0.0      0.089838    -1.916418   
BETA_TIME_BIKE  -6.823696           0.0      1.933551    -3.529100   
BETA_TIME_DRIVE -5.912480           0.0      2.764665    -2.138589   
BETA_TIME_PT    -3.215450           0.0      1.519538    -2.116071   
BETA_TIME_WALK  -8.443604           0.0      1.859213    -4.541494   
MOTOR            1.000000           1.0      0.495176     2.019485   
PRIVATIZED       1.000000           1.0      0.503879     1.984604   

                 Ro

In [12]:
general_stats_model_4 = results_model_4.getGeneralStatistics()
print(results_model_4.printGeneralStatistics())

Number of estimated parameters:	13
Number of free parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-3713.937
Likelihood ratio test for the init. model:	6435.07
Rho-square for the init. model:	0.464
Rho-square-bar for the init. model:	0.462
Akaike Information Criterion:	7453.874
Bayesian Information Criterion:	7538.598
Final gradient norm:	6.1511E+01
Nbr of threads:	8



In [13]:
LR_test = 2 * (results_model_4.data.logLike - results_model_2.data.logLike)
print("Log likelihood ratio test:", LR_test)
x_qhi = st.chi2.sf(LR_test, 4)
x_qhi = st.chi2.ppf(0.05, 4)
print("Critical value:", x_qhi)


Log likelihood ratio test: -1.5599962353007868e-06
Critical value: 0.7107230213973239


#### Cross nesting (final model)

In [14]:

# Define nest coefficients
MOTOR = Beta('MOTOR', 1, 1, None, 0)  # Nest parameter for motorized transport
PRIVATIZED = Beta('PRIVATIZED', 1, 1, None, 0)  # Nest parameter for non-motorized transport

# Define nests
alpha = 0.5
nest_motorized = MOTOR, {1: 0,
                         2: 0,
                         3: 1,
                         4: 1 - alpha}
nest_privatized = PRIVATIZED, {1: 1,
                               2: 1,
                               3: 0,
                               4: alpha}

# Combine nests into a list
nests = nest_motorized, nest_privatized

# Define the cross-nested logit model
crossnested_logit = models.logcnl(V, None, nests, travel_mode)

# Estimate the model
biogeme_4_crossnest = bio.BIOGEME(database, crossnested_logit)
biogeme_4_crossnest.modelName = 'Model_4_crossnest'
biogeme_4_crossnest.generateHtml = False  # Disable HTML file generation
biogeme_4_crossnest.generatePickle = False  # Disable PICKLE file generation
biogeme_4_crossnest.save_iterations = False  # Disable ITER file generation
results_model_4_cross = biogeme_4_crossnest.estimate()

# Print the estimation results
print(results_model_4_cross.getEstimatedParameters())

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


                    Value  Active bound  Rob. Std err  Rob. t-test  \
ASC_BIKE        -2.818309           0.0      0.341293    -8.257751   
ASC_BIKE_NOCAR  -0.795910           0.0      0.221952    -3.585954   
ASC_PT          -0.772469           0.0      0.048963   -15.776632   
ASC_PT_NOCAR     1.346258           0.0      0.186178     7.231026   
ASC_WALK         1.421322           0.0      0.196798     7.222249   
ASC_WALK_NOCAR   3.650459           0.0      0.342229    10.666717   
BETA_COST       -0.109882           0.0      0.016000    -6.867532   
BETA_TIME_BIKE  -4.878199           0.0      0.603889    -8.077975   
BETA_TIME_DRIVE -3.406401           0.0      0.467079    -7.292991   
BETA_TIME_PT    -1.746432           0.0      0.305824    -5.710577   
BETA_TIME_WALK  -7.724917           0.0      0.709253   -10.891619   
MOTOR            3.042011           0.0      0.783938     3.880422   
PRIVATIZED       1.000000           1.0      0.110539     9.046599   

                 Ro

In [15]:
general_stats_model_4_cross = results_model_4_cross.getGeneralStatistics()
print(results_model_4_cross.printGeneralStatistics())

Number of estimated parameters:	13
Number of free parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-3704.091
Likelihood ratio test for the init. model:	6454.762
Rho-square for the init. model:	0.466
Rho-square-bar for the init. model:	0.464
Akaike Information Criterion:	7434.181
Bayesian Information Criterion:	7518.905
Final gradient norm:	2.9799E+01
Nbr of threads:	8



In [16]:
LR_test = 2 * (results_model_4_cross.data.logLike - results_model_2.data.logLike)
print("Log likelihood ratio test:", LR_test)
p_val = st.chi2.sf(LR_test, 4)
print("p value:", p_val)
x_qhi = st.chi2.ppf(0.05, 4)
print("Critical value:", x_qhi)


Log likelihood ratio test: 19.692802102445967
p value: 0.0005741799399085548
Critical value: 0.7107230213973239


### Market share



#### Computing simulated market share

In [93]:
# size and weight of each strata
strata = {"females_44_less": len(df[(df['age']<=44)&(df['female']==1)]),
         "females_45_more": len(df[(df['age']>=45)&(df['female']==1)]),
         "males_44_less": len(df[(df['age']<=44)&(df['female']==0)]),
         "males_45_more": len(df[(df['age']>=45)&(df['female']==0)])}

total = {"females_44_less": 2841376,
         "females_45_more": 1519948,
         "males_44_less": 2926408,
         "males_45_more": 1379198}

total_population = sum(total.values())
total_sample = sum(strata.values())

weights = {k: total[k] * total_sample / (v * total_population) for k, v in strata.items()}
# k= type of people (female/man and age), v = number of the type k

In [94]:
strata

{'females_44_less': 1623,
 'females_45_more': 965,
 'males_44_less': 1517,
 'males_45_more': 895}

In [95]:
weights

{'females_44_less': 1.0099849525473574,
 'females_45_more': 0.9086698794546592,
 'males_44_less': 1.1128945356168978,
 'males_45_more': 0.889013383029116}

In [96]:
# insert weight as a new column
mask_ = {"females_44_less": (df['age']<=40)&(df['female']==1),
         "females_45_more": (df['age']>=41)&(df['female']==1),
         "males_44_less": (df['age']<=40)&(df['female']==0),
         "males_45_more": (df['age']>=41)&(df['female']==0)}
df['weight'] = 0
for k, v in mask_.items():
    df.loc[v, 'weight'] = weights[k]

C:\Users\Alexander\AppData\Local\Temp\ipykernel_15920\1433649319.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0099849525473574' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[v, 'weight'] = weights[k]


In [97]:
# market share simulated
database = db.Database('LPMC', df)

weight = Variable('weight')
prob_walk = models.cnl(V, None, nests, 1)
prob_cycling = models.cnl(V, None, nests, 2)
prob_pt = models.cnl(V, None, nests, 3)
prob_driving = models.cnl(V, None, nests, 4)
simulate = {
    'weight': weight,
    'prob.walk': prob_walk,
    'prob.cycling': prob_cycling,
    'prob.pt': prob_pt,
    'prob.driving': prob_driving
}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())
simulated_values

The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


,weight,prob.walk,prob.cycling,prob.pt,prob.driving
0,0.908670,9.944214e-03,0.027591,0.213024,0.749441
1,0.908670,5.801423e-01,0.021149,0.071568,0.327141
2,0.889013,1.083794e-05,0.013711,0.246321,0.739956
3,0.889013,1.248762e-08,0.008533,0.960095,0.031372
4,0.889013,4.333659e-06,0.011949,0.347323,0.640724
...,...,...,...,...,...
4995,0.889013,7.867356e-04,0.018227,0.296338,0.684648
4996,1.112895,5.085620e-07,0.012142,0.740644,0.247214
4997,1.112895,9.108640e-02,0.040436,0.106293,0.762185
4998,1.112895,3.857252e-03,0.039468,0.180142,0.776533


In [98]:
simulated_values['weighted walk'] = simulated_values['weight'] * simulated_values['prob.walk']
simulated_values['weighted cycling'] = simulated_values['weight'] * simulated_values['prob.cycling']
simulated_values['weighted pt'] = simulated_values['weight'] * simulated_values['prob.pt']
simulated_values['weighted driving'] = simulated_values['weight'] * simulated_values['prob.driving']

In [99]:
market_share_walk = simulated_values['weighted walk'].mean()
market_share_cycling = simulated_values['weighted cycling'].mean()
market_share_pt = simulated_values['weighted pt'].mean()
market_share_driving = simulated_values['weighted driving'].mean()

print(f"Market share of walk (simulated): {100*market_share_walk:.1f}%")
print(f"Market share of cycling(simulated): {100*market_share_cycling:.1f}%")
print(f"Market share of pt(simulated): {100*market_share_pt:.1f}%")
print(f"Market share of driving(simulated): {100*market_share_driving:.1f}%")

Market share of walk (simulated): 17.7%
Market share of cycling(simulated): 2.9%
Market share of pt(simulated): 35.2%
Market share of driving(simulated): 43.0%


In [100]:
help(simulate)

Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Built-in subclasses:
 |      StgDict
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if the dictionary has the specified key, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>va

#### Computing actual market share

In [101]:
#actual market share:

# weighted market shares using actual choices
mask_choice = {"females_44_less":{'walk': len(df[(df['age']<=44)&(df['female']==1)&(df['travel_mode']==1)]),
                                  'cycling':len(df[(df['age']<=44)&(df['female']==1)&(df['travel_mode']==2)]),
                                  'pt':len(df[(df['age']<=44)&(df['female']==1)&(df['travel_mode']==3)]),
                                  'driving':len(df[(df['age']<=44)&(df['female']==1)&(df['travel_mode']==4)])},
              "females_45_more": {'walk': len(df[(df['age']>=45)&(df['female']==1)&(df['travel_mode']==1)]),
                                  'cycling':len(df[(df['age']>=45)&(df['female']==1)&(df['travel_mode']==2)]),
                                  'pt':len(df[(df['age']>=45)&(df['female']==1)&(df['travel_mode']==3)]),
                                  'driving':len(df[(df['age']>=45)&(df['female']==1)&(df['travel_mode']==4)])},
              "males_44_less": {'walk': len(df[(df['age']<=44)&(df['female']==0)&(df['travel_mode']==1)]),
                                  'cycling':len(df[(df['age']<=44)&(df['female']==0)&(df['travel_mode']==2)]),
                                  'pt':len(df[(df['age']<=44)&(df['female']==0)&(df['travel_mode']==3)]),
                                  'driving':len(df[(df['age']<=44)&(df['female']==0)&(df['travel_mode']==4)])},
              "males_45_more": {'walk': len(df[(df['age']>=45)&(df['female']==0)&(df['travel_mode']==1)]),
                                  'cycling':len(df[(df['age']>=45)&(df['female']==0)&(df['travel_mode']==2)]),
                                  'pt':len(df[(df['age']>=45)&(df['female']==0)&(df['travel_mode']==3)]),
                                  'driving':len(df[(df['age']>=45)&(df['female']==0)&(df['travel_mode']==4)])}}

market_share_walk_weighted = sum([weights[k] * v['walk'] for k, v in mask_choice.items()])/total_sample
market_share_cycling_weighted = sum([weights[k] * v['cycling'] for k, v in mask_choice.items()])/total_sample
market_share_pt_weighted = sum([weights[k] * v['pt'] for k, v in mask_choice.items()])/total_sample
market_share_driving_weighted = sum([weights[k] * v['driving'] for k, v in mask_choice.items()])/total_sample

In [102]:
print(f"Weighted market share of walk: {100*market_share_walk_weighted:.1f}%")
print(f"Weighted market share of cycling: {100*market_share_cycling_weighted:.1f}%")
print(f"Weighted market share of pt: {100*market_share_pt_weighted:.1f}%")
print(f"Weighted market share of driving: {100*market_share_driving_weighted:.1f}%")

Weighted market share of walk: 18.1%
Weighted market share of cycling: 3.0%
Weighted market share of pt: 35.5%
Weighted market share of driving: 43.4%


#### Confidence interval

**Microsimulation Process:**

1. **Model Estimation:**
   - A choice model (like a multinomial logit model) is estimated using observed data.
   - The model estimates the probability $ P_n(i \mid x_n; \hat{\theta}) $ that individual $ n $ chooses alternative $ i $ based on their attributes $ x_n $ and the estimated parameters $ \hat{\theta} $.

2. **Simulation of Choices:**
   - For each individual in the sample, the choice model is used to simulate choices.
   - This typically involves drawing a random number and comparing it to the cumulative probability distribution of the choices to determine which alternative is selected.
   - Each simulation is repeated $ R $ times to capture the variability of choices due to the randomness in the model.

3. **Aggregation:**
   - The number of times each alternative is chosen across all simulations is counted to calculate the simulated number of individuals $ \hat{N}(i) $ choosing each alternative.
   - Aggregate market shares are then estimated by averaging over all simulations.

**Calculation of Aggregate Market Shares:**

1. **Number of Individuals Choosing Alternative $ i $:**
   - This is calculated as the average number of times alternative $ i $ is chosen across all $ R $ simulations for each individual.
   $$ \hat{N}(i) = \frac{1}{R} \sum_{n=1}^{N} \sum_{r=1}^{R} \hat{y}_{inr} $$

2. **Share of the Population Choosing Alternative $ i $:**
   - This is the proportion of the population that is estimated to choose alternative $ i $, averaged over all simulations.
   $$ \hat{W}(i) = \frac{1}{NR} \sum_{n=1}^{N} \sum_{r=1}^{R} \hat{y}_{inr} $$

**Calculation of Confidence Intervals:**

To calculate confidence intervals for the simulated market shares:

1. **Bootstrap Method:**
   - The bootstrap method involves resampling the simulated choice data with replacement and recalculating $ \hat{N}(i) $ and $ \hat{W}(i) $ for each resample.
   - This process is repeated many times (e.g., 1000 or more) to create an empirical distribution of the market shares.

2. **Confidence Interval Estimation:**
   - Confidence intervals are then derived from the empirical distribution of the bootstrapped market shares.
   - For example, the 95% confidence interval can be estimated using the 2.5th and 97.5th percentiles of the bootstrapped market shares.


In [103]:

# Number of bootstrap samples
n_bootstraps = 1000
confidence_level = 0.95

# Initialize an array to store the bootstrap market share estimates
bootstrap_market_shares = {
    'walk': [],
    'cycling': [],
    'pt': [],
    'driving': []
}

# Perform bootstrapping
for i in range(n_bootstraps):
    # Sample with replacement from the simulated_values
    sample = simulated_values.sample(n=len(simulated_values), replace=True)
    
    # Calculate weighted market shares for the bootstrap sample
    for mode in ['walk', 'cycling', 'pt', 'driving']:
        market_share = (sample['weight'] * sample[f'prob.{mode}']).mean()
        bootstrap_market_shares[mode].append(market_share)

# Calculate the confidence intervals
lower_bound = (1 - confidence_level) / 2
upper_bound = 1 - lower_bound

market_share_confidence_intervals = {}
for mode in ['walk', 'cycling', 'pt', 'driving']:
    lower = np.percentile(bootstrap_market_shares[mode], lower_bound * 100)
    upper = np.percentile(bootstrap_market_shares[mode], upper_bound * 100)
    market_share_confidence_intervals[mode] = (lower, upper)

# Print the confidence intervals
for mode, interval in market_share_confidence_intervals.items():
    print(f"{mode.capitalize()} market share 95% confidence interval: {100*interval[0]:.1f}% - {100*interval[1]:.1f}%")


Walk market share 95% confidence interval: 17.1% - 18.4%
Cycling market share 95% confidence interval: 2.8% - 2.9%
Pt market share 95% confidence interval: 34.4% - 36.1%
Driving market share 95% confidence interval: 42.2% - 43.7%


### Forecasting

We consider two scenarios:
1. An increase of 1.50 GBP for car users
2. A decrease of the public transport charge of 20%


#### Predicted market share in both cases:

In [104]:
#predicted market share in case of increase of car costs:

V4_s1 = (BETA_COST * (cost_driving_total + 1.5) + BETA_TIME_DRIVE * dur_driving)
V_s1 = {1: V1, 2: V2, 3: V3, 4: V4_s1}
prob_walk = models.cnl(V_s1, None, nests, 1)
prob_cycling = models.cnl(V_s1, None, nests, 2)
prob_pt = models.cnl(V_s1, None, nests, 3)
prob_driving_scenario1 = models.cnl(V_s1, None, nests, 4)
simulate = {
    'weight': weight,
    'prob.walk': prob_walk,
    'prob.cycling': prob_cycling,
    'prob.pt': prob_pt,
    'prob.driving': prob_driving_scenario1
}

biosim_s1 = bio.BIOGEME(database, simulate) #using database defined in market share
simulated_values_s1 = biosim_s1.simulate(results_model_4_cross.getBetaValues())

simulated_values_s1['weighted walk'] = simulated_values_s1['weight'] * simulated_values_s1['prob.walk']
simulated_values_s1['weighted cycling'] = simulated_values_s1['weight'] * simulated_values_s1['prob.cycling']
simulated_values_s1['weighted pt'] = simulated_values_s1['weight'] * simulated_values_s1['prob.pt']
simulated_values_s1['weighted driving'] = simulated_values_s1['weight'] * simulated_values_s1['prob.driving']

market_share_walk_s1 = simulated_values_s1['weighted walk'].mean()
market_share_cycling_s1 = simulated_values_s1['weighted cycling'].mean()
market_share_pt_s1 = simulated_values_s1['weighted pt'].mean()
market_share_driving_s1 = simulated_values_s1['weighted driving'].mean()

print('Scenario 1: increase car cost by 1.5 pounds')
print(f"Market share of walk: {100*market_share_walk_s1:.2f}%")
print(f"Market share of cycling: {100*market_share_cycling_s1:.2f}%")
print(f"Market share of pt: {100*market_share_pt_s1:.2f}%")
print(f"Market share of driving: {100*market_share_driving_s1:.2f}%")

The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Scenario 1: increase car cost by 1.5 pounds
Market share of walk: 18.46%
Market share of cycling: 3.08%
Market share of pt: 38.81%
Market share of driving: 38.45%


In [105]:
#predicted market share in case of decrease in public transport

V3_s2 = ASC_PT * car_available + ASC_PT_NOCAR * (1 - car_available) + BETA_COST * cost_transit*0.08 + BETA_TIME_PT * dur_pt_total
V_s2 = {1: V1, 2: V2, 3: V3_s2, 4: V4}
prob_walk = models.cnl(V_s2, None, nests, 1)
prob_cycling = models.cnl(V_s2, None, nests, 2)
prob_pt = models.cnl(V_s2, None, nests, 3)
prob_driving_scenario1 = models.cnl(V_s2, None, nests, 4)
simulate = {
    'weight': weight,
    'prob.walk': prob_walk,
    'prob.cycling': prob_cycling,
    'prob.pt': prob_pt,
    'prob.driving': prob_driving_scenario1
}

biosim_s2 = bio.BIOGEME(database, simulate) #using database defined in market share
simulated_values_s2 = biosim_s2.simulate(results_model_4_cross.getBetaValues())

simulated_values_s2['weighted walk'] = simulated_values_s2['weight'] * simulated_values_s2['prob.walk']
simulated_values_s2['weighted cycling'] = simulated_values_s2['weight'] * simulated_values_s2['prob.cycling']
simulated_values_s2['weighted pt'] = simulated_values_s2['weight'] * simulated_values_s2['prob.pt']
simulated_values_s2['weighted driving'] = simulated_values_s2['weight'] * simulated_values_s2['prob.driving']

market_share_walk_s2 = simulated_values_s2['weighted walk'].mean()
market_share_cycling_s2 = simulated_values_s2['weighted cycling'].mean()
market_share_pt_s2 = simulated_values_s2['weighted pt'].mean()
market_share_driving_s2 = simulated_values_s2['weighted driving'].mean()

print('Scenario 2: decrease public transport costs by 20%')
print(f"Market share of walk: {100*market_share_walk_s2:.2f}%")
print(f"Market share of cycling: {100*market_share_cycling_s2:.2f}%")
print(f"Market share of pt: {100*market_share_pt_s2:.2f}%")
print(f"Market share of driving: {100*market_share_driving_s2:.2f}%")

The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Scenario 2: decrease public transport costs by 20%
Market share of walk: 17.34%
Market share of cycling: 2.73%
Market share of pt: 39.03%
Market share of driving: 39.69%


When wanting to decrease the share of car, we should consider the first scenario, as the simulated market share of driving is **39.13%**. In the second scenario, the car market share is **39.35%**, thus higher.

### Highest pt revenue 

We want to check in which scenario the public transportation revenue is the highest. To do so, we need to compute the revenue in all 3 cases (no changes, increase in car costs, decrease in pt costs).

In [106]:
database = db.Database('LPMC', df)
weight = Variable('weight')

In [107]:
#no change in policy

prob_walk = models.cnl(V, None, nests, 1)
prob_cycling = models.cnl(V, None, nests, 2)
prob_pt = models.cnl(V, None, nests, 3)
prob_driving_scenario1 = models.cnl(V, None, nests, 4)

simulate = {
    'weight': weight,
    'revenues PT': prob_pt * cost_transit

}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())

print(f"Public transport revenues (no changes in policy): {simulated_values['revenues PT'].sum()}")


The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Public transport revenues (no changes in policy): 3439.7365663069395


In [108]:
#scenario 1 (increase in car costs)

prob_walk = models.cnl(V_s1, None, nests, 1)
prob_cycling = models.cnl(V_s1, None, nests, 2)
prob_pt = models.cnl(V_s1, None, nests, 3)
prob_driving_scenario1 = models.cnl(V_s1, None, nests, 4)

simulate = {
    'weight': weight,
    'revenues PT': prob_pt * cost_transit

}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())

print(f"Public transport revenues (scenario 1 policy): {simulated_values['revenues PT'].sum()}")

The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Public transport revenues (scenario 1 policy): 3707.2121875182793


In [109]:
#scenario 2 (decrease in pt costs)

prob_walk = models.cnl(V_s2, None, nests, 1)
prob_cycling = models.cnl(V_s2, None, nests, 2)
prob_pt = models.cnl(V_s2, None, nests, 3)
prob_driving_scenario1 = models.cnl(V_s2, None, nests, 4)

simulate = {
    'weight': weight,
    'revenues PT': prob_pt * cost_transit

}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())

print(f"Public transport revenues (scenario 2 policy): {simulated_values['revenues PT'].sum()}")

The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Public transport revenues (scenario 2 policy): 3972.919982635551


The **second scenario** gives the highest revenues for public transport.

### Average value of time

We want to compute the average VOT for both car and public transportation (in GBP/hour). To do so, we use the following formula:

$$ \text{(VOT)}_\text{i} = \frac{\partial  \text{(Utility)}_i}{\partial \text{(duration)}_i} : \frac{\partial  \text{(Utility)}_i}{\partial \text{(cost)}_i} , i \in \{car,pt\} $$


In [110]:
V_pt = models.cnl(V, None, nests, 3)
V_driving = models.cnl(V, None, nests, 4)

#idk why it doesn't work without defining both of the utilities upper ????

vot_pt = Derive(V_pt, 'dur_pt_total') / Derive(V_pt, 'cost_transit')
vot_car = Derive(V_driving, 'dur_driving') / Derive(V_driving, 'cost_driving_total')

simulate = {
    'weight': weight,
    'WTP PT time': vot_pt,
    'WTP CAR time': vot_car,
}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())



The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


In [111]:
print(f"Average value of time for public transport: {(simulated_values['weight']*simulated_values['WTP PT time']).mean()}  GBP/hour")
print(f"Average value of time for car: {(simulated_values['weight']*simulated_values['WTP CAR time']).mean()} GBP/hour")

Average value of time for public transport: 15.702796494892324  GBP/hour
Average value of time for car: 30.628176945517996 GBP/hour


### Direct and cross aggregate elasticities

Now we need to compute the direct and cross elasticites of car costs and public transport costs. 

The **direct price elasticity** for the car is the percent change in pt change resulting from a 1% change in car costs. The formula is given by:

$$ E^{car}_{pt} =  \frac{(cost_{transit})}{(cost_{driving_total})} \cdot \frac{\partial  (cost_{driving_total})}{\partial (cost_{transit})}$$

The **cross price elasticity** is given by the following formula:



In [112]:
prob_pt = models.cnl(V, None, nests, 3)
prob_driving = models.cnl(V, None, nests, 4)

#direct elasticities 
direct_elas_pt_cost = Derive(prob_pt, 'cost_transit') * cost_transit / prob_pt
direct_elas_driving_cost = Derive(prob_driving, 'cost_driving_total') * cost_driving_total / prob_driving

simulate = {
    'weight': weight,
    'prob.driving': prob_driving,
    'prob.pt': prob_pt,
    'direct_elas_pt_cost': direct_elas_pt_cost,
    'direct_elas_driving_cost': direct_elas_driving_cost
}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())

The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


In [113]:
simulated_values['numerator_pt_cost'] = simulated_values['weight'] * simulated_values['prob.pt'] * simulated_values['direct_elas_pt_cost']
simulated_values['numerator_driving_cost'] = simulated_values['weight'] * simulated_values['prob.driving'] * simulated_values['direct_elas_driving_cost']
simulated_values['denominator_pt_cost'] = simulated_values['weight'] * simulated_values['prob.pt']
simulated_values['denominator_driving_cost'] = simulated_values['weight'] * simulated_values['prob.driving']

In [114]:
#aggregate elasticities

agg_elast_pt_cost = simulated_values['numerator_pt_cost'].sum()/simulated_values['denominator_pt_cost'].sum()
agg_elast_driving_cost = simulated_values['numerator_driving_cost'].sum()/simulated_values['denominator_driving_cost'].sum()

print(f"Elasticity of public transport cost: {agg_elast_pt_cost}")
print(f"Elasticity of public driving cost: {agg_elast_driving_cost}")

Elasticity of public transport cost: -0.11245905931957516
Elasticity of public driving cost: -0.0719072180238916
